In [1]:
# Cell 1: Install Dependencies
!pip install -q gradio google-generativeai markdown weasyprint
!apt-get update -qq
!apt-get install -qq weasyprint

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.1/851.1 kB 17.0 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Extracting templates from packages: 100%
Preconfiguring packages ...
Selecting previously unselected package libtext-iconv-perl.
(Reading database ... 126435 files and directories currently installed.)
Preparing to unpack .../00-libtext-iconv-perl_1.7-7build3_amd64.deb ...
Unpacking libtext-iconv-perl (1.7-7build3) ...
Selecting previously unselected package dictionaries-common.
Preparing to unpack .../01-dictionaries-common_1.28.14_all.deb ...
Adding 'diversion of /usr/share/dict/words to /usr/share/dict/words.pre-dictionaries-common by dictionaries-common'
Unpackin

In [2]:
# Cell 2: Import Libraries and Setup
import gradio as gr
import google.generativeai as genai
from markdown import markdown
from weasyprint import HTML
import os
import getpass
from IPython.display import display, Markdown

print("🔑 Setting up Gemini API Key...")

# Option 1: Try to get from Colab Secrets first
api_key = os.environ.get('GEMINI_API_KEY')

# Option 2: If not found in secrets, prompt for input
if not api_key:
    print("🤔 GEMINI_API_KEY not found in Colab Secrets.")
    print("📝 Please enter your Gemini API key below:")
    api_key = getpass.getpass("Enter your Gemini API key: ")

    # Validate that something was entered
    if not api_key or api_key.strip() == "":
        raise ValueError("❌ No API key provided. Please enter a valid Gemini API key.")

# Option 3: Save to environment for this session
os.environ['GEMINI_API_KEY'] = api_key

# Configure Gemini
try:
    genai.configure(api_key=api_key)
    print("✅ Gemini API configured successfully!")

    # Test the connection
    try:
        model = genai.GenerativeModel('gemini-1.5-flash')
        response = model.generate_content("Say 'Hello World!'")
        print("🧪 API connection test: SUCCESS")
        print(f"   Response: {response.text[:50]}...")
    except Exception as test_error:
        print(f"⚠️  API connection test failed: {test_error}")
        print("   The API key might be invalid or you may have exceeded your quota.")

except Exception as config_error:
    raise ValueError(f"❌ Failed to configure Gemini API: {config_error}")

🔑 Setting up Gemini API Key...
🤔 GEMINI_API_KEY not found in Colab Secrets.
📝 Please enter your Gemini API key below:
Enter your Gemini API key: ··········
✅ Gemini API configured successfully!
🧪 API connection test: SUCCESS
   Response: Hello World!
...


In [3]:
# Cell 3: Define Functions (COMPLETE UPDATED VERSION)
import os
import tempfile
import time

def create_prompt(resume_string: str, jd_string: str) -> str:
    """
    Creates a detailed prompt for AI-powered resume optimization based on a job description.
    """
    return f"""
You are a professional resume optimization expert specializing in tailoring resumes to specific job descriptions. Your goal is to optimize my resume and provide actionable suggestions for improvement to align with the target role.

### Guidelines:
1. **Relevance**:
   - Prioritize experiences, skills, and achievements **most relevant to the job description**.
   - Remove or de-emphasize irrelevant details to ensure a **concise** and **targeted** resume.
   - Limit work experience section to 2-3 most relevant roles
   - Limit bullet points under each role to 2-3 most relevant impacts

2. **Action-Driven Results**:
   - Use **strong action verbs** and **quantifiable results** (e.g., percentages, revenue, efficiency improvements) to highlight impact.

3. **Keyword Optimization**:
   - Integrate **keywords** and phrases from the job description naturally to optimize for ATS (Applicant Tracking Systems).

4. **Additional Suggestions** *(If Gaps Exist)*:
   - If the resume does not fully align with the job description, suggest:
     1. **Additional technical or soft skills** that I could add to make my profile stronger.
     2. **Certifications or courses** I could pursue to bridge the gap.
     3. **Project ideas or experiences** that would better align with the role.

5. **Formatting**:
   - Output the tailored resume in **clean Markdown format**.
   - Include an **"Additional Suggestions"** section at the end with actionable improvement recommendations.

---

### Input:
- **My resume**:
{resume_string}

- **The job description**:
{jd_string}

---

### Output:
1. **Tailored Resume**:
   - A resume in **Markdown format** that emphasizes relevant experience, skills, and achievements.
   - Incorporates job description **keywords** to optimize for ATS.
   - Uses strong language and is no longer than **one page**.

2. **Additional Suggestions** *(if applicable)*:
   - List **skills** that could strengthen alignment with the role.
   - Recommend **certifications or courses** to pursue.
   - Suggest **specific projects or experiences** to develop.
"""

def get_resume_response(prompt: str, model: str = "gemini-1.5-flash", temperature: float = 0.7) -> str:
    """
    Sends a resume optimization prompt to Google's Gemini API and returns the optimized resume response.
    """
    # Check if using placeholder key
    api_key = os.environ.get('GEMINI_API_KEY', '')
    if api_key == "placeholder-key-for-testing":
        return """⚠️ **API Key Required** ⚠️

Please provide a valid Gemini API key to optimize your resume!

**How to get your API key:**
1. Go to: https://aistudio.google.com/app/apikey
2. Sign in with your Google account
3. Click "Create API key"
4. Copy the key (starts with 'AIza...')
5. In Colab: Left sidebar → 🔑 Key icon → Add secret:
   - Name: `GEMINI_API_KEY`
   - Value: Your API key
   - Toggle "Notebook access" ON
6. Restart runtime and re-run Cell 2

**Until then, you can still:**
- Test the UI interface
- Export to PDF (if you manually edit the resume)
- View the layout and functionality"""

    try:
        # Initialize Gemini model
        model_obj = genai.GenerativeModel(
            model_name=model,
            generation_config={"temperature": temperature}
        )

        # Make API call
        response = model_obj.generate_content(
            contents=prompt,
            generation_config={"max_output_tokens": 2000}
        )

        # Extract and return response
        if response.text:
            return response.text
        else:
            return "❌ No response received from Gemini API. Please check your API key and quota."

    except Exception as e:
        return f"❌ API Error: {str(e)}\n\n💡 Please check:\n• Your API key is valid\n• You have sufficient quota\n• Internet connection is stable"

def read_resume_file(resume_input):
    """
    Helper function to read resume file, handling both file objects and NamedString paths.
    """
    try:
        # Case 1: If it's a file-like object with .read() method
        if hasattr(resume_input, 'read'):
            return resume_input.read().decode('utf-8')

        # Case 2: If it's a NamedString (newer Gradio versions) - contains file path
        elif hasattr(resume_input, 'name') and isinstance(resume_input.name, str):
            with open(resume_input.name, 'r', encoding='utf-8') as f:
                return f.read()

        # Case 3: If it's just a string path
        elif isinstance(resume_input, str) and os.path.exists(resume_input):
            with open(resume_input, 'r', encoding='utf-8') as f:
                return f.read()

        # Case 4: If it's a string content (direct paste)
        elif isinstance(resume_input, str):
            return resume_input

        else:
            raise ValueError(f"Unsupported resume input type: {type(resume_input)}")

    except Exception as e:
        raise ValueError(f"Failed to read resume file: {str(e)}")

def process_resume(resume_input, jd_string):
    """
    Process a resume file against a job description to create an optimized version.
    FIXED: Now handles NamedString objects from Gradio.
    """
    # Check if resume input is provided
    if resume_input is None:
        error_msg = "❌ **No resume uploaded!**\n\nPlease upload a .md or .txt file with your resume content."
        return error_msg, "", error_msg

    # Check if job description is provided
    if not jd_string or jd_string.strip() == "":
        error_msg = "❌ **No job description provided!**\n\nPlease paste the job description in the text box."
        return error_msg, "", error_msg

    try:
        # Read resume content using helper function
        resume_string = read_resume_file(resume_input).strip()

        if not resume_string:
            error_msg = "❌ **Empty resume file!**\n\nPlease upload a file with actual resume content."
            return error_msg, "", error_msg

        # Debug: Print first 100 chars to verify
        print(f"📄 Resume preview ({len(resume_string)} chars): {resume_string[:100]}...")

        # Create prompt
        prompt = create_prompt(resume_string, jd_string)

        # Generate response
        response_string = get_resume_response(prompt)

        # If it's an error message, return it
        if response_string.startswith("⚠️") or response_string.startswith("❌"):
            return response_string, "", response_string

        # Parse response to separate resume and suggestions
        if "## Additional Suggestions" in response_string:
            response_list = response_string.split("## Additional Suggestions", 1)
            new_resume = response_list[0].strip()
            suggestions = "## Additional Suggestions\n\n" + response_list[1].strip()
        else:
            new_resume = response_string.strip()
            suggestions = "✅ No additional suggestions needed - your resume is well-aligned with the job!"

        print(f"✅ Processing successful! Generated {len(new_resume)} chars of optimized resume.")
        return new_resume, new_resume, suggestions

    except Exception as e:
        error_msg = f"❌ **Processing Error**: {str(e)}\n\n💡 Please ensure:\n• Resume file is valid .md or .txt format\n• Job description is complete\n• File is not corrupted"
        print(f"❌ Processing failed: {str(e)}")
        return error_msg, "", error_msg

def export_resume(new_resume):
    """
    Convert a markdown resume to PDF format and save it.
    FIXED: Now saves to /content/ for easy download with timestamped filename.
    """
    if not new_resume or new_resume.strip() == "":
        return "❌ **Nothing to export!**\n\nPlease optimize a resume first or enter content in the editor."

    try:
        # Create a permanent file in /content/ with a descriptive timestamped name
        timestamp = int(time.time())
        output_pdf_file = f"/content/optimized_resume_{timestamp}.pdf"

        # Convert Markdown to HTML
        html_content = markdown(new_resume)

        # Create styled HTML for better PDF output
        html_with_style = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <meta charset="UTF-8">
            <title>Optimized Resume - {time.strftime('%Y-%m-%d')}</title>
            <style>
                body {{
                    font-family: 'Helvetica', 'Arial', sans-serif;
                    font-size: 11pt;
                    line-height: 1.4;
                    color: #333;
                    max-width: 8.5in;
                    margin: 0.5in auto;
                    padding: 0;
                }}
                h1 {{
                    font-size: 24pt;
                    font-weight: bold;
                    margin: 0 0 10px 0;
                    color: #2c3e50;
                    border-bottom: 2px solid #3498db;
                    padding-bottom: 5px;
                }}
                h2 {{
                    font-size: 14pt;
                    font-weight: bold;
                    margin: 20px 0 8px 0;
                    color: #34495e;
                }}
                h3 {{
                    font-size: 12pt;
                    font-weight: bold;
                    margin: 15px 0 5px 0;
                    color: #2c3e50;
                }}
                p, ul {{
                    margin: 5px 0;
                }}
                ul {{
                    padding-left: 20px;
                    margin-left: 0;
                }}
                li {{
                    margin: 3px 0;
                }}
                .contact {{
                    font-size: 10pt;
                    margin: 5px 0;
                    color: #666;
                }}
                .date {{
                    float: right;
                    font-style: italic;
                    color: #666;
                    font-size: 10pt;
                }}
                .clear {{ clear: both; }}
                a {{ color: #3498db; text-decoration: none; }}
                a:hover {{ text-decoration: underline; }}
                .skills {{
                    display: flex;
                    flex-wrap: wrap;
                    gap: 8px;
                    margin: 5px 0;
                }}
                .skill-tag {{
                    background: #ecf0f1;
                    padding: 2px 8px;
                    border-radius: 12px;
                    font-size: 9pt;
                    color: #2c3e50;
                }}
            </style>
        </head>
        <body>
            {html_content}
        </body>
        </html>
        """

        # Convert HTML to PDF
        HTML(string=html_with_style).write_pdf(output_pdf_file)

        # Get file size and preview
        file_size = os.path.getsize(output_pdf_file) / 1024  # KB
        preview_text = new_resume[:50].replace('\n', ' ').replace('[', '').replace(']', '')

        # Return success message with clear download instructions
        return f"""
<div style="background: #d4edda; padding: 15px; border-radius: 8px; border-left: 4px solid #28a745;">
<h3>✅ **PDF Export Successful!** 🎉</h3>

**📁 File Location:** <code>{output_pdf_file}</code>

**📥 **EASY DOWNLOAD STEPS:** 📥**
<ol style="margin: 10px 0; padding-left: 20px;">
  <li><strong>Click the 📁 FILES icon</strong> (folder) in the <strong>left sidebar</strong></li>
  <li><strong>Look for</strong> <code>optimized_resume_{timestamp}.pdf</code> in the main <code>/content/</code> folder</li>
  <li><strong>Right-click the PDF</strong> → <strong>"Download"</strong></li>
  <li><strong>OR drag the file</strong> directly to your desktop!</li>
</ol>

**🔍 File Details:**
<ul style="margin: 10px 0; padding-left: 20px;">
  <li><strong>Name:</strong> <code>optimized_resume_{timestamp}.pdf</code></li>
  <li><strong>Size:</strong> ~{file_size:.1f} KB</li>
  <li><strong>Preview:</strong> {preview_text}...</li>
  <li><strong>Date:</strong> {time.strftime('%Y-%m-%d %H:%M:%S')}</li>
</ul>

**💾 All exported PDFs are saved in <code>/content/</code> and won't be deleted!**
</div>

**💡 Pro Tip:** You can keep multiple versions by exporting multiple times - each gets a unique timestamp!
        """

    except Exception as e:
        error_msg = f"❌ **PDF Export Failed**: {str(e)}"
        print(f"❌ Export failed: {str(e)}")
        return f"""
<div style="background: #f8d7da; padding: 15px; border-radius: 8px; border-left: 4px solid #dc3545;">
<h3>{error_msg}</h3>

**💡 Troubleshooting:**
<ul style="margin: 10px 0; padding-left: 20px;">
  <li>• Ensure the resume content is valid Markdown</li>
  <li>• Check if WeasyPrint is properly installed (re-run Cell 1)</li>
  <li>• Try simplifying complex formatting in the resume</li>
  <li>• Ensure you have write permissions in <code>/content/</code></li>
</ul>

**🔄 Try again after fixing the issue above.**
</div>
        """

print("✅ **Functions defined successfully!** (Complete Updated Version)")
print("   • create_prompt() - Creates optimization prompts")
print("   • get_resume_response() - Handles Gemini API calls")
print("   • read_resume_file() - Handles NamedString file reading")
print("   • process_resume() - Main processing function (FIXED)")
print("   • export_resume() - PDF export to /content/ (FIXED)")
print("\n🚀 **Ready for Cell 4 - Gradio UI!**")
print("\n📝 **Key Fixes Applied:**")
print("   • ✅ NamedString compatibility for Gradio 4.0+")
print("   • ✅ Permanent PDF storage in /content/")
print("   • ✅ Timestamped filenames for multiple exports")
print("   • ✅ Enhanced error handling and user feedback")
print("   • ✅ Better PDF styling and formatting")

✅ **Functions defined successfully!** (Complete Updated Version)
   • create_prompt() - Creates optimization prompts
   • get_resume_response() - Handles Gemini API calls
   • read_resume_file() - Handles NamedString file reading
   • process_resume() - Main processing function (FIXED)
   • export_resume() - PDF export to /content/ (FIXED)

🚀 **Ready for Cell 4 - Gradio UI!**

📝 **Key Fixes Applied:**
   • ✅ NamedString compatibility for Gradio 4.0+
   • ✅ Permanent PDF storage in /content/
   • ✅ Timestamped filenames for multiple exports
   • ✅ Enhanced error handling and user feedback
   • ✅ Better PDF styling and formatting


In [4]:
# Cell 4: Create and Launch Gradio UI
print("🎨 Building the Resume Optimizer Interface...")

# Create the Gradio interface
with gr.Blocks(title="Resume Optimizer", theme=gr.themes.Soft()) as app:
    # Header
    gr.Markdown("""
    # 📄 **Resume Optimizer**
    *Powered by Google Gemini*

    **Upload your resume → Paste a job description → Get an optimized version!**

    *Tip: Works best with .md format resumes, but .txt files work too!*
    """)

    # API Key Status
    api_key_status = os.environ.get('GEMINI_API_KEY', '')
    if api_key_status == "placeholder-key-for-testing":
        gr.Markdown("""
        ⚠️ **API Key Status: Test Mode**
        *Resume optimization requires a valid Gemini API key. Get one at [Google AI Studio](https://aistudio.google.com/app/apikey)*
        """, elem_classes="alert-warning")
    else:
        gr.Markdown("✅ **API Key Status: Ready!**", elem_classes="alert-success")

    # Main input section
    with gr.Row():
        with gr.Column(scale=1):
            # Resume upload
            resume_input = gr.File(
                label="📁 **Upload Your Resume**",
                file_types=[".md", ".txt"],
                type="filepath",
                elem_id="resume-upload"
            )

            # Job description input
            jd_input = gr.Textbox(
                label="📋 **Paste Job Description**",
                lines=10,
                placeholder="""Paste the full job description here...

Example: "We are looking for a Senior Software Engineer with 3+ years of experience in Python, React, and AWS. Responsibilities include..."

💡 Tip: Include all requirements, skills, and responsibilities from the job posting.""",
                elem_id="job-description"
            )

            # Optimize button
            optimize_btn = gr.Button(
                "✨ **Optimize My Resume**",
                variant="primary",
                size="lg",
                elem_id="optimize-btn"
            )

        with gr.Column(scale=1):
            gr.Markdown("### 📝 **Sample Resume Format**")
            gr.Code("""
# John Doe
john.doe@email.com | (123) 456-7890 | linkedin.com/in/johndoe

## Professional Summary
Results-driven software engineer with 5+ years experience...

## Experience
### Senior Developer, TechCorp (2020-Present)
- Led development of scalable web applications
- Improved system performance by 40% through optimization

## Skills
Python, JavaScript, AWS, Docker, Git
            """, language="markdown", lines=15, show_label=False)

    # Results section
    with gr.Accordion("📊 **Optimization Results**", open=True):
        with gr.Row():
            with gr.Column(scale=2):
                # Optimized resume preview
                output_resume_md = gr.Markdown(
                    label="✨ **Optimized Resume**",
                    value="**Upload a resume and job description to see your optimized version here!**",
                    elem_id="resume-preview"
                )

                # Suggestions
                output_suggestions = gr.Markdown(
                    label="💡 **Actionable Suggestions**",
                    value="**Optimization will generate personalized suggestions here.**",
                    elem_id="suggestions"
                )

            with gr.Column(scale=1):
                gr.Markdown("### ✏️ **Edit & Export**")
                output_resume = gr.Textbox(
                    label="📝 **Editable Resume**",
                    lines=15,
                    interactive=True,
                    placeholder="Your optimized resume will appear here. Edit as needed, then export to PDF!",
                    elem_id="editable-resume"
                )

                export_btn = gr.Button("📄 **Export to PDF**", variant="secondary")
                export_result = gr.Markdown(
                    label="📥 **Export Status**",
                    value="Click 'Export to PDF' after editing your resume!",
                    elem_id="export-status"
                )

    # Footer
    gr.Markdown("""
    ---
    *Built with ❤️ using Gradio & Google Gemini | Uploads are processed securely in your Colab session*
    """)

    # Event handlers
    def handle_optimize(resume_file, job_desc):
        """Handle the optimize button click"""
        try:
            # Process resume and get (new_resume, new_resume, suggestions)
            result = process_resume(resume_file, job_desc)
            if len(result) == 3:  # Ensure tuple has 3 elements
                new_resume_md, new_resume_text, suggestions = result
                return new_resume_md, new_resume_text, suggestions
            else:
                error_msg = "❌ **Processing Error**: Unexpected output format from process_resume"
                return error_msg, "", error_msg
        except Exception as e:
            error_msg = f"❌ **Unexpected Error**: {str(e)}"
            return error_msg, "", error_msg

    def handle_export(resume_text):
        """Handle the export button click"""
        try:
            if not resume_text or resume_text.strip() == "":
                return "❌ **Nothing to export!** Please optimize a resume first or enter content in the editor."
            result = export_resume(resume_text)
            return result
        except Exception as e:
            return f"❌ **Export Error**: {str(e)}"

    # Bind events
    optimize_btn.click(
        fn=handle_optimize,
        inputs=[resume_input, jd_input],
        outputs=[output_resume_md, output_resume, output_suggestions]
    )

    export_btn.click(
        fn=handle_export,
        inputs=[output_resume],
        outputs=[export_result]
    )

    # Auto-populate and sync editable field with optimized resume
    def update_editable(resume_md, resume_text, suggestions):
        """Sync the editable field with the optimized resume text"""
        return resume_text if resume_text else "No content to edit yet."

    optimize_btn.click(
        fn=update_editable,
        inputs=[output_resume_md, output_resume, output_suggestions],
        outputs=[output_resume]
    )

# Launch the application
print("🌐 Launching Gradio interface...")
print("⏳ This may take 30-60 seconds...")

try:
    # Launch with public sharing enabled
    demo = app.launch(
        share=True,
        debug=True,
        show_error=True,
        server_port=7860,
        favicon_path=None
    )

    print("✅ **Success!** Gradio interface launched!")
    print(f"🔗 **Public URL**: {demo}")
    print("\n📱 **How to use:**")
    print("1. Click the public URL above")
    print("2. Upload your resume (.md or .txt)")
    print("3. Paste a job description")
    print("4. Click 'Optimize My Resume'")
    print("5. Edit if needed, then export to PDF!")
    print("\n⏰ **Note**: Public URL expires in ~72 hours")
    print("💾 **PDFs** are saved in Colab's `/content/` folder")

except Exception as launch_error:
    print(f"❌ **Launch Error**: {launch_error}")
    print("\n🔧 **Troubleshooting**:")
    print("• Wait a moment and try running Cell 4 again")
    print("• Check if port 7860 is available")
    print("• Try restarting runtime and re-running all cells")
    print("• Ensure you have a stable internet connection")

    # Fallback: Try local launch
    print("\n🔄 **Trying local launch**...")
    try:
        app.launch(share=False, debug=True)
        print("✅ Local interface launched! (Only visible in Colab)")
    except Exception as local_error:
        print(f"❌ Local launch also failed: {local_error}")
        print("💡 Please restart runtime and try again")

🎨 Building the Resume Optimizer Interface...
🌐 Launching Gradio interface...
⏳ This may take 30-60 seconds...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f246764208aa5aaed5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


📄 Resume preview (2497 chars): # Shaw Talebi  
**Email**: [shawhintalebi@gmail.com](mailto:shawhintalebi@gmail.com)  
**Homepage**:...
✅ Processing successful! Generated 3952 chars of optimized resume.


DEBUG:fontTools.ttLib.ttFont:Reading 'maxp' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'maxp' table
DEBUG:fontTools.subset.timer:Took 0.002s to load 'maxp'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'maxp'
INFO:fontTools.subset:maxp pruned
DEBUG:fontTools.ttLib.ttFont:Reading 'cmap' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'cmap' table
DEBUG:fontTools.ttLib.ttFont:Reading 'post' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'post' table
DEBUG:fontTools.subset.timer:Took 0.009s to load 'cmap'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'cmap'
INFO:fontTools.subset:cmap pruned
INFO:fontTools.subset:fpgm dropped
INFO:fontTools.subset:prep dropped
INFO:fontTools.subset:cvt  dropped
INFO:fontTools.subset:kern dropped
DEBUG:fontTools.subset.timer:Took 0.000s to load 'post'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'post'
INFO:fontTools.subset:post pruned
INFO:fontTools.subset:GPOS dropped
INFO:fontTools.subset:GSUB dropped
DEBUG:f

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f246764208aa5aaed5.gradio.live
✅ **Success!** Gradio interface launched!
🔗 **Public URL**: 

📱 **How to use:**
1. Click the public URL above
2. Upload your resume (.md or .txt)
3. Paste a job description
4. Click 'Optimize My Resume'
5. Edit if needed, then export to PDF!

⏰ **Note**: Public URL expires in ~72 hours
💾 **PDFs** are saved in Colab's `/content/` folder
